In [4]:
from selenium import webdriver      
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import urllib.request
import time

from datetime import date
import pandas as pd
date_today = date.today()

driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get('https://shop.telfar.net/collections/t-shirts')

soup = BeautifulSoup(driver.page_source,'html.parser')

In [20]:
product_url = []
picture_url = []
timestamp = []
brand = []
product_type = []
sales_price = []
price = []
name = []

for i in soup.find_all('div',{'class':'product grid__item one-whole slide-up-animation animated'}):
    
    timestamp.append(date_today)
    brand.append('Telfar')
    product_type.append(None)
    
    product_url.append("https://shop.telfar.net" + i.find('div',{'class':'supports-js'}).a.get('href'))
    picture_url.append("https:" + str(i.find('div',{'class':'supports-js'}).a.img.get('data-src')).replace('{width}', '600'))
    
    name.append(i.find('div',{'class':'product__title text-center'}).a.text)
    sales_price.append(i.find('div',{'class':'product__prices text-center'}).span.text.replace('Regular price','').strip())
    price.append(None)

# Second Layer

In [28]:
material = []
product_description = []
picture_name = []
color = []
counter = 1

driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
for link in product_url:

    driver.get(link)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    product_description.append(soup.find('div',{'itemprop':'description'}).find('div',{'class':'column'}).text.strip())
    
    color.append(None)
    material.append(None)
    picture_name.append('telfar_' + str(counter) + '.jpg') # change variable
    counter += 1

In [29]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", "picture_name"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Telfar,None,FW20 World Tour T-Shirt,Short sleeve crewneck T-shirt made from 100% ...,None,$ 65.00,None,None,https://shop.telfar.net/collections/t-shirts/p...,https://cdn.shopify.com/s/files/1/0880/7204/pr...,2020-12-09,telfar_1.jpg
1,Telfar,None,Dance T-Shirt - Charcoal,Short sleeve crewneck T-shirt made from 100% ...,None,$ 110.00,None,None,https://shop.telfar.net/collections/t-shirts/p...,https://cdn.shopify.com/s/files/1/0880/7204/pr...,2020-12-09,telfar_2.jpg
2,Telfar,None,Dance T-Shirt - Off Black,Short sleeve crewneck T-shirt made from 100% ...,None,$ 110.00,None,None,https://shop.telfar.net/collections/t-shirts/p...,https://cdn.shopify.com/s/files/1/0880/7204/pr...,2020-12-09,telfar_3.jpg
3,Telfar,None,60S Logo T-Shirt,Short sleeve crewneck T-shirt made from 100% ...,None,$ 88.00,None,None,https://shop.telfar.net/collections/t-shirts/p...,https://cdn.shopify.com/s/files/1/0880/7204/pr...,2020-12-09,telfar_4.jpg
4,Telfar,None,Mirror T-Shirt,Short sleeve crewneck T-shirt made from 100% ...,None,$ 98.00,None,None,https://shop.telfar.net/collections/t-shirts/p...,https://cdn.shopify.com/s/files/1/0880/7204/pr...,2020-12-09,telfar_5.jpg


In [30]:
second_pass.to_csv("telfar.csv", index=False, encoding = 'utf-8-sig')

In [32]:
import os
import requests as req

os.mkdir('telfar')
i = 1
for index, link in enumerate(picture_url):
    img_data = req.get(link).content
    with open('telfar/'+ "telfar_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1